# Histology

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
origen = "lung_scc"
destino = "pulmon_carcinoma"

# Listar archivos en la carpeta de origen
archivos = os.listdir(origen)

# Iterar sobre los archivos en la carpeta de origen
for archivo in archivos:
    ruta_origen = os.path.join(origen, archivo)
    ruta_destino = os.path.join(destino, archivo)
    
    # Verificar si el archivo es una imagen (puedes agregar más extensiones si es necesario)
    if archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        # Verificar si es un archivo (evitar carpetas u otros elementos)
        if os.path.isfile(ruta_origen):
            # Leer la imagen con OpenCV
            imagen = cv.imread(ruta_origen)
            if imagen is not None:
                # Convertir la imagen a escala de grises
                nueva_imagen = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
                # Guardar la copia con el mismo nombre en la carpeta destino
                cv.imwrite(ruta_destino, nueva_imagen)
                print(f'Imagen copiada y procesada: {archivo}')
            else:
                print(f'Error al leer: {archivo}')
        else:
            print(f'Omitido (no es un archivo): {archivo}')
    else:
        print(f'No es una imagen: {archivo}')

In [12]:
# Parámetros
dataset_dir = "C:\\Users\\Dragut\\Desktop\\bobio\\Samsung\\PF"
batch_size = 32
img_size = (224, 224)  # Ajusta el tamaño deseado
validation_split = 0.2  # 20% de los datos para validación
seed = 123  # Semilla para reproducibilidad

# Carga del dataset para entrenamiento
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="training",        # Usamos la parte de entrenamiento
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",  # O 'int' según tus necesidades
    shuffle=True
)

# Carga del dataset para validación
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="validation",      # Usamos la parte de validación
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True
)


# Verificar etiquetas asignadas
class_names = train_ds.class_names  # Debería mostrar ['cats', 'dogs']
print("Clases asignadas:", class_names)

# Verificar tamaños de los datasets
train_size = tf.data.experimental.cardinality(train_ds).numpy()
val_size = tf.data.experimental.cardinality(val_ds).numpy()

print(f"Entrenamiento: {train_size} batches")
print(f"Validación: {val_size} batches")

Found 15000 files belonging to 3 classes.
Using 12000 files for training.
Found 15000 files belonging to 3 classes.
Using 3000 files for validation.
Clases asignadas: ['pulmon_adenocarcinoma', 'pulmon_benigno', 'pulmon_carcinoma']
Entrenamiento: 375 batches
Validación: 94 batches


In [13]:
input_shape = (224, 224, 3)
num_classes = 3

In [19]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'], loss="categorical_crossentropy")

In [20]:
early_stopping = EarlyStopping(
    monitor='val_loss',       # Monitorea la pérdida de validación
    patience=5,               # Si no mejora en 5 épocas consecutivas, se detiene
    restore_best_weights=True # Restaura los mejores pesos durante el entrenamiento
)

with tf.device('/GPU:0'):
    history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[early_stopping])

Epoch 1/50
375/375 [==============================] - 67s 162ms/step - loss: 0.2004 - accuracy: 0.9194 - val_loss: 6.3145 - val_accuracy: 0.3577
Epoch 2/50
375/375 [==============================] - 59s 156ms/step - loss: 0.0410 - accuracy: 0.9863 - val_loss: 4.6321 - val_accuracy: 0.5457
Epoch 3/50
375/375 [==============================] - 59s 156ms/step - loss: 0.0214 - accuracy: 0.9921 - val_loss: 5.9198 - val_accuracy: 0.5513
Epoch 4/50
375/375 [==============================] - 59s 156ms/step - loss: 0.0254 - accuracy: 0.9925 - val_loss: 0.1099 - val_accuracy: 0.9733
Epoch 5/50
375/375 [==============================] - 58s 155ms/step - loss: 0.0157 - accuracy: 0.9956 - val_loss: 0.1259 - val_accuracy: 0.9737
Epoch 6/50
375/375 [==============================] - 58s 154ms/step - loss: 0.0111 - accuracy: 0.9958 - val_loss: 0.4679 - val_accuracy: 0.9260
Epoch 7/50
375/375 [==============================] - 58s 153ms/step - loss: 0.0191 - accuracy: 0.9942 - val_loss: 0.5227 - val_ac

In [21]:
#Evaluamos el modelo
loss, accuracy = model.evaluate(val_ds)
print(f"Accuracy: {accuracy}")

94/94 [==============================] - 5s 49ms/step - loss: 1.7611e-04 - accuracy: 1.0000
Accuracy: 1.0


In [ ]:
# Crear las listas de etiquetas verdaderas y predicciones
y_true_list = []
y_pred_list = []

for images, labels in val_ds:
    y_true_list.append(labels.numpy())  # Convertimos las etiquetas a numpy
    y_pred_probs = model.predict(images)  # Predecimos las probabilidades
    y_pred_labels = np.argmax(y_pred_probs, axis=1)  # Convertimos las probabilidades a 0 o 1
    y_pred_list.append(y_pred_labels)  # Guardamos las predy_test_clases = np.argmax(y_true, axis=1)icciones
    
y_test_clases = np.argmax(y_true, axis=1)

# Unir las listas de etiquetas verdaderas y predicciones
y_true = np.concatenate(y_true_list)
y_test_clases = np.argmax(y_true, axis=1)
y_pred = np.concatenate(y_pred_list)

1/1 [==============================] - 0s 23ms/step


In [33]:
# Crear la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Visualizarla con seaborn
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Verdadero")
plt.title("Matriz de Confusión")
plt.show()

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [30]:
y_pred

array([0, 1, 1, ..., 0, 2, 2], dtype=int64)